This FINAL homework recaps the Spark ML library:

0) Download the "Rain in Australia" dataset from Kaggle (it is also attached to this assigbnment): https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

1) In a Jupyter Notebook, write a SparkML script that uses a Decision Tree Classifier to predict the RainTomorrow target varible

2) Split the data 80/20 train/test, using a seed of 12345

3) Use transformers to remove unnecessary columns (use your best judgement) and convert categorical variables into one-hot encoded variables

4) Use a parameter grid to determine the best parameters for: impurity - gini, entropy maxBins - 5, 10, 15 minInfoGain - 0.0, 0.2, 0.4 maxDepth - 3, 5, 7

5) Cross-validate with 4 folds

6) Use a pipeline to encapsulate all steps

7) Print the parameters from the best model selected

8) Calculate and print the Area under ROC Curve and Area under Precision-Recall Curve scores for your training and test data sets (these are built-in metrics, you do not need to calculate anything by hand)

Your script should be clean of all the testing and exploration and should only contain the necessary code to satisfy the above conditions

## library load

In [8]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import RFormula
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.mllib.evaluation import BinaryClassificationMetrics

## create data frame

In [9]:
spark = SparkSession.builder.getOrCreate()
df=spark.read.option("header", "true").csv("weatherAUS.csv")
df.show(5)

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RainTomorrow|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|2008-12-01|  Albury|   13.4|   22.9|     0.6|         NA|      NA|          W|           44|         W|       WNW|          20|          24|         71|         22|     1007.7|     1007.1|       8|      NA|   16.9|   21.8|       No|          No|
|2008-12-02|

## I saw some NA block in the csv, so I drop them

In [10]:
col_list=df.toPandas().columns.values.tolist()
df.select ("RainTomorrow").distinct().collect()
df_clean=df.where(df["RainTomorrow"] != "NA")


## using Rformula to read data
http://zwmiller.com/projects/spark_ml_example_part2.html

In [11]:
formula = RFormula(formula="RainTomorrow ~ .",featuresCol="features",labelCol="label")
output = formula.fit(df_clean).transform(df_clean)

### A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values.
### Class for indexing categorical feature columns in a dataset of Vector.
### follow the requirement to split the data


In [12]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(output)
featureIndexer = (VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(output))
(trainingData, testData) = output.randomSplit([0.8, 0.2], seed=12345)





22/04/27 23:44:30 WARN DAGScheduler: Broadcasting large task binary with size 1494.4 KiB


### DecisionTreeClassifier created
### pipline created

In [13]:
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

In [14]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

### setting up the require vaule

In [15]:
grid = ParamGridBuilder().addGrid(dt.impurity, ["gini", "entropy"]).addGrid(dt.maxBins, [5, 10, 15]).addGrid(dt.minInfoGain, [0.0, 0.2, 0.4]).addGrid(dt.maxDepth, [3, 5, 7]).build()

In [16]:
cv= CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=grid, numFolds=4)

## training

In [10]:
cvModel = cv.fit(trainingData)

22/04/27 10:58:32 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/04/27 10:58:35 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/04/27 10:58:38 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/04/27 10:58:44 WARN MemoryStore: Not enough space to cache rdd_275_3 in memory! (computed 67.5 MiB so far)
22/04/27 10:58:44 WARN BlockManager: Persisting block rdd_275_3 to disk instead.
22/04/27 10:58:44 WARN MemoryStore: Not enough space to cache rdd_275_0 in memory! (computed 103.7 MiB so far)
22/04/27 10:58:44 WARN BlockManager: Persisting block rdd_275_0 to disk instead.
22/04/27 10:58:44 WARN MemoryStore: Not enough space to cache rdd_275_1 in memory! (computed 103.7 MiB so far)
22/04/27 10:58:44 WARN BlockManager: Persisting block rdd_275_1 to disk instead.
22/04/27 10:58:45 WARN MemoryStore: Not enough space to cache rdd_275_2 in memory! (computed 355.0 MiB so far)
22/04/27 10:58:45 WARN BlockManager: Persisting block rd

22/04/27 10:59:24 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/04/27 10:59:24 WARN MemoryStore: Not enough space to cache rdd_368_2 in memory! (computed 103.7 MiB so far)
22/04/27 10:59:24 WARN MemoryStore: Not enough space to cache rdd_368_3 in memory! (computed 103.7 MiB so far)
22/04/27 10:59:24 WARN MemoryStore: Not enough space to cache rdd_368_1 in memory! (computed 103.7 MiB so far)
22/04/27 10:59:24 WARN MemoryStore: Not enough space to cache rdd_368_0 in memory! (computed 103.7 MiB so far)
22/04/27 10:59:26 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/04/27 10:59:26 WARN MemoryStore: Not enough space to cache rdd_368_0 in memory! (computed 103.7 MiB so far)
22/04/27 10:59:26 WARN MemoryStore: Not enough space to cache rdd_368_2 in memory! (computed 103.7 MiB so far)
22/04/27 10:59:26 WARN MemoryStore: Not enough space to cache rdd_368_3 in memory! (computed 103.7 MiB so far)
22/04/27 10:59:26 WARN MemoryStore: Not enough spac

22/04/27 11:00:05 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/04/27 11:00:05 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/04/27 11:00:06 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/04/27 11:00:06 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/04/27 11:00:06 WARN MemoryStore: Not enough space to cache rdd_561_3 in memory! (computed 67.5 MiB so far)
22/04/27 11:00:06 WARN BlockManager: Persisting block rdd_561_3 to disk instead.
22/04/27 11:00:07 WARN MemoryStore: Not enough space to cache rdd_561_0 in memory! (computed 103.7 MiB so far)
22/04/27 11:00:07 WARN BlockManager: Persisting block rdd_561_0 to disk instead.
22/04/27 11:00:07 WARN MemoryStore: Not enough space to cache rdd_561_2 in memory! (computed 103.7 MiB so far)
22/04/27 11:00:07 WARN BlockManager: Persisting block rdd_561_2 to disk instead.
22/04/27 11:00:07 WARN MemoryStore: Not enough space to cache rdd_561_1 in memory

22/04/27 11:00:40 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/04/27 11:00:40 WARN MemoryStore: Not enough space to cache rdd_672_2 in memory! (computed 103.7 MiB so far)
22/04/27 11:00:40 WARN MemoryStore: Not enough space to cache rdd_672_3 in memory! (computed 103.7 MiB so far)
22/04/27 11:00:40 WARN MemoryStore: Not enough space to cache rdd_672_1 in memory! (computed 103.7 MiB so far)
22/04/27 11:00:40 WARN MemoryStore: Not enough space to cache rdd_672_0 in memory! (computed 103.7 MiB so far)
22/04/27 11:00:42 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/04/27 11:00:43 WARN MemoryStore: Not enough space to cache rdd_672_1 in memory! (computed 103.7 MiB so far)
22/04/27 11:00:43 WARN MemoryStore: Not enough space to cache rdd_672_2 in memory! (computed 103.7 MiB so far)
22/04/27 11:00:43 WARN MemoryStore: Not enough space to cache rdd_672_0 in memory! (computed 103.7 MiB so far)
22/04/27 11:00:43 WARN MemoryStore: Not enough spac

22/04/27 11:01:14 WARN MemoryStore: Not enough space to cache rdd_807_0 in memory! (computed 355.0 MiB so far)
22/04/27 11:01:14 WARN BlockManager: Persisting block rdd_807_0 to disk instead.
22/04/27 11:01:15 WARN MemoryStore: Not enough space to cache rdd_807_3 in memory! (computed 44.0 MiB so far)
22/04/27 11:01:17 WARN MemoryStore: Not enough space to cache rdd_807_2 in memory! (computed 103.7 MiB so far)
22/04/27 11:01:17 WARN MemoryStore: Not enough space to cache rdd_807_0 in memory! (computed 155.9 MiB so far)
22/04/27 11:01:17 WARN MemoryStore: Not enough space to cache rdd_807_1 in memory! (computed 355.2 MiB so far)
22/04/27 11:01:19 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/04/27 11:01:19 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/04/27 11:01:19 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/04/27 11:01:20 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/04/27 11:01:20 WAR

...

22/04/27 11:32:41 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/04/27 11:32:41 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/04/27 11:32:41 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/04/27 11:32:42 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/04/27 11:32:43 WARN MemoryStore: Not enough space to cache rdd_7962_1 in memory! (computed 67.5 MiB so far)
22/04/27 11:32:43 WARN BlockManager: Persisting block rdd_7962_1 to disk instead.
22/04/27 11:32:43 WARN MemoryStore: Not enough space to cache rdd_7962_2 in memory! (computed 103.7 MiB so far)
22/04/27 11:32:43 WARN BlockManager: Persisting block rdd_7962_2 to disk instead.
22/04/27 11:32:43 WARN MemoryStore: Not enough space to cache rdd_7962_3 in memory! (computed 103.7 MiB so far)
22/04/27 11:32:43 WARN BlockManager: Persisting block rdd_7962_3 to disk instead.
22/04/27 11:32:44 WARN MemoryStore: Not enough space to cache rdd_7962_0 in

22/04/27 11:33:21 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/04/27 11:33:21 WARN MemoryStore: Not enough space to cache rdd_8073_2 in memory! (computed 103.7 MiB so far)
22/04/27 11:33:21 WARN MemoryStore: Not enough space to cache rdd_8073_0 in memory! (computed 103.7 MiB so far)
22/04/27 11:33:21 WARN MemoryStore: Not enough space to cache rdd_8073_3 in memory! (computed 103.7 MiB so far)
22/04/27 11:33:21 WARN MemoryStore: Not enough space to cache rdd_8073_1 in memory! (computed 103.7 MiB so far)
22/04/27 11:33:23 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/04/27 11:33:24 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/04/27 11:33:24 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/04/27 11:33:25 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/04/27 11:33:25 WARN MemoryStore: Not enough space to cache rdd_8120_2 in memory! (computed 67.5 MiB so far)
22/04/27 11

22/04/27 11:33:59 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/04/27 11:33:59 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/04/27 11:33:59 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/04/27 11:34:00 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/04/27 11:34:00 WARN MemoryStore: Not enough space to cache rdd_8243_3 in memory! (computed 67.5 MiB so far)
22/04/27 11:34:00 WARN BlockManager: Persisting block rdd_8243_3 to disk instead.
22/04/27 11:34:01 WARN MemoryStore: Not enough space to cache rdd_8243_2 in memory! (computed 103.7 MiB so far)
22/04/27 11:34:01 WARN BlockManager: Persisting block rdd_8243_2 to disk instead.
22/04/27 11:34:01 WARN MemoryStore: Not enough space to cache rdd_8243_1 in memory! (computed 103.7 MiB so far)
22/04/27 11:34:01 WARN BlockManager: Persisting block rdd_8243_1 to disk instead.
22/04/27 11:34:01 WARN MemoryStore: Not enough space to cache rdd_8243_0 in

22/04/27 11:34:37 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/04/27 11:34:37 WARN MemoryStore: Not enough space to cache rdd_8383_3 in memory! (computed 103.7 MiB so far)
22/04/27 11:34:37 WARN MemoryStore: Not enough space to cache rdd_8383_2 in memory! (computed 103.7 MiB so far)
22/04/27 11:34:37 WARN MemoryStore: Not enough space to cache rdd_8383_1 in memory! (computed 103.7 MiB so far)
22/04/27 11:34:37 WARN MemoryStore: Not enough space to cache rdd_8383_0 in memory! (computed 103.7 MiB so far)
22/04/27 11:34:39 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/04/27 11:34:40 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/04/27 11:34:40 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/04/27 11:34:41 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/04/27 11:34:41 WARN MemoryStore: Not enough space to cache rdd_8424_0 in memory! (computed 67.5 MiB so far)
22/04/27 11

22/04/27 11:35:20 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/04/27 11:35:21 WARN MemoryStore: Not enough space to cache rdd_8524_3 in memory! (computed 67.5 MiB so far)
22/04/27 11:35:21 WARN BlockManager: Persisting block rdd_8524_3 to disk instead.
22/04/27 11:35:21 WARN MemoryStore: Not enough space to cache rdd_8524_2 in memory! (computed 103.7 MiB so far)
22/04/27 11:35:21 WARN BlockManager: Persisting block rdd_8524_2 to disk instead.
22/04/27 11:35:21 WARN MemoryStore: Not enough space to cache rdd_8524_1 in memory! (computed 103.7 MiB so far)
22/04/27 11:35:21 WARN BlockManager: Persisting block rdd_8524_1 to disk instead.
22/04/27 11:35:22 WARN MemoryStore: Not enough space to cache rdd_8524_0 in memory! (computed 355.0 MiB so far)
22/04/27 11:35:22 WARN BlockManager: Persisting block rdd_8524_0 to disk instead.
22/04/27 11:35:22 WARN MemoryStore: Not enough space to cache rdd_8524_3 in memory! (computed 44.0 MiB so far)
22/04/27 11:35:24 WARN Memor

In [11]:
test_metric = evaluator.evaluate(cvModel.transform(testData))

22/04/27 11:37:25 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


In [12]:
test_metric

0.7981008444584603

## Print the parameters from the best model selected

In [26]:
best_Model = cvModel.bestModel
for i in range(len(best_Model.stages)):
    print(best_Model.stages[i])

StringIndexerModel: uid=StringIndexer_722d5a54cc00, handleInvalid=error
VectorIndexerModel: uid=VectorIndexer_fca0cf2b4740, numFeatures=8018, handleInvalid=error
DecisionTreeClassificationModel: uid=DecisionTreeClassifier_69aa8a77b962, depth=7, numNodes=179, numClasses=2, numFeatures=8018


In [28]:
java_model = best_Model.stages[-1]._java_obj
{param.name: java_model.getOrDefault(java_model.getParam(param.name)) 
    for param in grid[0]}

{'impurity': 'gini', 'maxBins': 5, 'minInfoGain': 0.0, 'maxDepth': 7}

In [29]:
predictionAndLabels = best_Model.transform(testData)

## Calculate and print the Area under ROC Curve and Area under Precision-Recall Curve scores

In [30]:

metrics = BinaryClassificationMetrics(predictionAndLabels.select("prediction","indexedLabel").rdd)


print("Area under PR = %s" % metrics.areaUnderPR)


print("Area under ROC = %s" % metrics.areaUnderROC)

22/04/27 11:50:09 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/04/27 11:50:11 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


Area under PR = 0.4831531580955708
Area under ROC = 0.6112066519249517
